In [16]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
import subprocess
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.ttfonts import TTFont
import random
import locale
locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

In [17]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
import subprocess
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.ttfonts import TTFont
registerFont(TTFont('Calibri', 'Calibri Regular.ttf')) # Just some font imports

In [18]:
def SalaryFiller(can):

    basic = (random.choice([14000, 15000, 17000, 18000, 21000, 22000]))
    can.drawString(353, 445, f'{basic:n}')

    hra = (random.choice([9000, 9500, 10000, 10500, 11000, 11500]))
    can.drawString(353, 432, f'{hra:n}')

    conveyance = (random.choice([700, 800, 900, 1000]))
    can.drawString(353, 419, str(conveyance))

    dearness_allowance = (random.choice([450, 500, 600, 700, 800]))
    can.drawString(353, 406, str(dearness_allowance))

    overtime = (random.choice([100, 125, 150, 175, 200]))
    can.drawString(353, 393, f'{overtime:n}')

    subtotal = basic + hra + overtime + dearness_allowance + conveyance
    can.drawString(353, 343, f'{subtotal:n}')

    state_insurance = ((int)(7 * basic) / 10000) * 100
    can.drawString(353, 321, f'{state_insurance:n}')

    pf = ((int)(3 * basic) / 10000) * 100
    can.drawString(353, 299, f'{pf:n}')

    NetSalary = subtotal * (0.96) - (state_insurance + pf)
    can.drawString(355, 241, f'{NetSalary:n}')

def PDFcreator(name, surname, date_of_birth, gender, original="original.pdf", destination="destination.pdf"):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    can.setFont("Calibri", 11)
    can.drawString(158, 620, name+" " +surname)
    can.drawString(360, 620, gender)
    can.drawString(360, 606, date_of_birth)
    SalaryFiller(can)
    can.save()

    # move to the beginning of the StringIO buffer
    packet.seek(0)

    # create a new PDF with Reportlab
    new_pdf = PdfFileReader(packet)
    # read your existing PDF
    existing_pdf = PdfFileReader(open(original, "rb"))
    output = PdfFileWriter()
    # add the "watermark" (which is the new pdf) on the existing page
    page = existing_pdf.getPage(0)
    page.mergePage(new_pdf.getPage(0))
    output.addPage(page)
    # finally, write "output" to a real file
    outputStream = open(destination, "wb")
    output.write(outputStream)
    outputStream.close()

#     bashCommand = "open " + destination
#     process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
#     output, error = process.communicate()

In [19]:
PDFcreator("Harry", "Potter","31/07/1980","M","salary_slip_format_basic_edited.pdf","Auror_MoM.pdf")

In [20]:
import json
from pathlib import Path
from datetime import datetime

# date_obj = datetime.strptime(date_input, '%d-%b-%Y')
# print date_obj.strftime('%d-%m-%Y')

In [44]:
def fileiterator(filename):
    # Opening JSON file
    f = open(filename)
    # returns JSON object as a dictionary
    text_data = json.load(f)
    # Iterating through the json list
    data=text_data["_via_img_metadata"]
    for items in data:
        common_dict=dict()
        for attributes in data[items]["regions"]:
            common_dict[attributes["region_attributes"]["field_name"]]=attributes["region_attributes"]["value"]
        dob=(common_dict["birth_date"])

#         dob=dob.replace("Okt","Oct")#grc_passport
#         date_obj=datetime.strptime(dob, '%d %b %y')#grc_passport
#         dob=date_obj.strftime('%d/%m/%Y')#grc_passport

        date_obj=datetime.strptime(dob, '%d.%m.%Y')
#         date_obj=datetime.strptime(dob, '%d-%m-%Y')
        dob=date_obj.strftime('%d/%m/%Y')

        gender=common_dict["gender"]#.split(" / ")[1]
        
        name=common_dict["name"].title()
        
        surname=common_dict["surname"].title()#+" "+common_dict["surname_second"].title()
        
        print(name+" "+surname+" "+gender+" "+dob)

        foldername=filename.split(".")[0]
        
        Path(foldername).mkdir(parents=True, exist_ok=True)
        
        itemname=items.split(".")[0]
        
        print(itemname)
        PDFcreator(name, surname,dob,gender,"salary_slip_format_basic.pdf",foldername+"/"+itemname+".pdf")
    f.close()

# fileiterator("templates/annotations/alb_id.json")
# fileiterator("templates/annotations/aze_passport.json") #gender contains prefix, date of birth has . instead of / ??
# fileiterator("templates/annotations/esp_id.json") # 2 surnames, date with spaces
# fileiterator("templates/annotations/est_id.json") # gender contains prefix
# fileiterator("templates/annotations/fin_id.json") # dob split by " " and .
# fileiterator("templates/annotations/grc_passport.json") # weird style for October and date month year, and names as well!
# fileiterator("templates/annotations/lva_passport.json") # date of birth was strange - . after year!
# fileiterator("templates/annotations/rus_internalpassport.json")# weird names, no_eng!
# fileiterator("templates/annotations/srb_passport.json") # gender split by spaces and slashes
fileiterator("templates/annotations/svk_id.json")

Refik Mikuš M 22/05/1955
00
Laren Zugan M 18/06/1980
01
Leander Barbalić M 12/12/1994
02
Christopher Beqiri M 02/08/1970
03
Djevad Majko M 08/04/1992
04
Ziba Hudin F 08/06/1994
05
Mersiha Kerštanj F 09/01/1974
06
Ornela Kovčan F 20/03/1964
07
Midheta Pergarec F 23/12/1961
08
Zvijezdana Kužnik F 11/05/1988
09
Damjan Tatarević M 21/04/1984
10
Šačir Ćesko M 15/03/1966
11
Pane Zmagaj M 02/03/1998
12
Ajiz Zarnec M 18/07/1986
13
Vehbija Tehovnik M 15/08/1995
14
Ljuba Budimir F 03/03/1974
15
Valčka Holzknecht F 24/04/1997
16
Jolan Troger F 13/06/1983
17
Alja Ržaničanin F 16/10/1965
18
Tatle Hrenić F 28/02/1982
19
Muhamet Beširović M 28/06/1985
20
Feručo Globočnik M 07/03/1988
21
Senad Smerdu M 22/04/1995
22
Đulijano Lorkovič M 18/12/1954
23
Kasim Bajič M 11/12/1989
24
Armela Kurnjek F 13/01/1964
25
Olena Mihič F 07/01/1998
26
Alessandra Šklebek F 16/12/1978
27
Jaroslava Poredoš F 27/11/1982
28
Gorjana Filovski F 04/09/1991
29
Zoro Luci M 15/01/1976
30
Dardan Verstovšek M 22/07/2000
31
Boris V